## first

In [1]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json
from langchain.chat_models import ChatOpenAI
# from dataloader import to_Collection



llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo")  

reflect_template = """
    Question: {question}
    Answer_choices: {answer_choices}

    {cot_trigger} {cot}
    {answer_extraction} {answer}
    
    Reflection: {reflection_prompt}
    """
reflect_prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt'], template=reflect_template)
reflect_chain = LLMChain(llm=llm, prompt=reflect_prompt_template,output_key="reflection")

#{instruction}
extraction_template = """

    Question: {question}
    Answer_choices: {answer_choices}

    {cot_trigger} {cot}
    {answer_extraction} {answer}
    
    Reflection: {reflection_prompt}
    {reflection}

    {reflect_answer_extraction}
    """
    #Get reflection
ans_prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt','reflection','reflect_answer_extraction'], template=extraction_template)
reflect_answer_chain = LLMChain(llm=llm, prompt=ans_prompt_template,output_key="reflection_answer")

    # This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
reflect_overall_chain = SequentialChain(chains=[reflect_chain, reflect_answer_chain],input_variables=["question","answer_choices","cot_trigger","answer_extraction",'cot','answer','reflection_prompt','reflect_answer_extraction'],
        output_variables=["reflection", "reflection_answer"],
        verbose=True)

In [1]:
from cot import Collection
ts_hard = Collection.from_json('ts_hard_v1.json')

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robertpraas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
ts_hard = ts_hard.select('all',1)
ts_hard

| Name           | Train   | Valid   | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       | 1       | -      |
| worldtree      | -       | -       | 1      |
| strategy_qa    | 1       | -       | -      |
| open_book_qa   | -       | -       | 1      |
| med_qa         | -       | -       | 1      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'medmc_qa', 'pubmed_qa', 'qed', 'svamp']

In [29]:
ts_hard.unload_datasets(['strategy_qa','open_book_qa','med_qa'])

In [30]:
ts_hard

| Name           | Train   | Valid   | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       | 1       | -      |
| worldtree      | -       | -       | 1      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [31]:
input_dict = {
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, the answer is", 
    'answer':"", 
    'cot': "", 
    'reflection_prompt':"Attempt to give a better Answer than the one before",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',
    'model_name':"gpt-3.5-turbo"
}
ts_hard.metareason_flexible(chain=reflect_overall_chain,input_dict=input_dict)

Generating commonsense_qa...


> Entering new SequentialChain chain...

> Finished chain.
Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [32]:
ts_hard

| Name           | Train   | Valid   | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       | 1       | -      |
| worldtree      | -       | -       | 1      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [2]:
ts_hard['strategy_qa']

DatasetDict({
    train: Dataset({
        features: ['id', 'ref_id', 'question', 'type', 'choices', 'context', 'cot', 'answer', 'generated_cot', 'feedback'],
        num_rows: 11
    })
})

SyntaxError: illegal target for annotation (3890503272.py, line 1)

In [ ]:
'reflection_prompt':"Double check this",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?'

 'reflection_prompt':"Critique the reasoning above",
    'reflect_answer_extraction':'Based on the reasoning and the reflection, what is the final answer? (A-D)?'

  'reflection_prompt':"Do you have any reason to believe that the reasoning or the answer might be wrong? Answer with one word Yes or No",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?'

 'reflection_prompt':"Improve the step-by-step thinking",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?'

'reflection_prompt':"Is there any irrelevant or incorrect information in the Answer",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',

 'reflection_prompt':"The goal is to correct the Answer if needed, let's think step by step",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',

'reflection_prompt':"Attempt to give a better Answer than the one before",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',

    'reflection_prompt':"Attempt to give a better Answer than the one before",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',